# PCO. Création d'une BDD relationnelle démonstrative 
## Compétences C2. C3. C4. C10.

![IllustrationDatabase](Ressources_NB\IllustrationDatabase.png)

Ce fichier présente l'ensemble des scripts de création de BDD, de table, de requêtes d'import/export des données si la base de données était construite directement à partir des fichiers CSV mis à disposition sur le site Kaggle. Le nombre de données étant conséquent, cette BDD "échantillon" est réalisée pour la validation des compétences associées. Une autre BDD a été créée spécialement pour la démonstration sur l'application et ayant l'avantage d'effectuer le traitement des requêtes plus rapidement.

## Compétences
* **C2.** Concevoir une base de données analytique avec l’approche orientée requêtes en vue de la mise à disposition des données pour un traitement analytique ou d’intelligence artificielle.
* **C3.** Programmer l’import de données initiales nécessaires au projet en base de données, afin de les rendre exploitables par un tiers, dans un langage de programmation adapté et à partir de la stratégie de nettoyage des données préalablement définie.
* **C4.** Préparer les données disponibles depuis la base de données analytique en vue de leur utilisation par les algorithmes d’intelligence artificielle.
* **C10.** Concevoir une base de données relationnelle à l’aide de méthodes standards de modélisation de données.

## Librairies

In [2]:
# Librairies standards de data-analyses:
    
import numpy as np
from numpy import set_printoptions
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
from scipy.stats import norm, skew
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ttest_ind


# sklearn modules Preprocessing:

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# sklearn modules Model Selection:

from sklearn import tree, linear_model, neighbors
from sklearn import naive_bayes, ensemble, discriminant_analysis, gaussian_process
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier

import xgboost
from xgboost import XGBClassifier


# sklearn modules Model Evaluation & Improvement:
    
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import learning_curve

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

from sklearn import feature_selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn import model_selection

from sklearn import metrics
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score, fbeta_score
from sklearn.metrics import make_scorer, recall_score, log_loss, matthews_corrcoef
from sklearn.metrics import average_precision_score
from sklearn.pipeline import make_pipeline


# Réseaux de neurones Tensorflow Keras:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers


# Librairies standards de data-visualisation:

import seaborn as sn
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib 
%matplotlib inline
color = sn.color_palette()
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches
from IPython.display import display
pd.options.display.max_columns = None
from dateutil.relativedelta import relativedelta
import datetime
from datetime import date, datetime
import re


# Importation IA:
import joblib


# Filtre warnings:

import warnings 
warnings.filterwarnings("ignore")


# Coloration des graphes Seaborn:

colors = ['#440154', '#482475','#414487','#355F8D','#2A788E','#21918C','#22A884','#44BF70','#7AD151','#BDDF26','#FDE725']
red = '#fd8181'
blue = '#8198fd'

## Fonctions

In [3]:
# Fonction détection des valeurs manquantes:
def find_NaN(dataset):
    NaN = dataset.isnull().sum().to_frame('Valeurs Manquantes')
    NaN = NaN.loc[NaN['Valeurs Manquantes']>0]
    NaN['Pourcentage'] = [str(round((i*100)/len(dataset),1)) for i in NaN['Valeurs Manquantes']]
    NaN['Pourcentage'] = [i+' %' for i in NaN['Pourcentage']]
    if NaN.empty:
        return print(f'Aucune valeur manquante n\'est détectée sur ce dataset')
    else:
        return NaN
    
# Fonction transformation date int(YYYYMMDD) en str(YYYY-MM-DD):    
def Transform_date(serie):
    Liste = []
    for i in serie:
        i = str(i)
        y = i[:4]
        m = i[4:6]
        d = i[6:]
        i = f'{d}-{m}-{y}'
        Liste.append(i)

    return Liste

# Fonction transformation date str(YYYY-MM-DD) en int(YYYYMMDD):    
def Format_intdate(serie):
    Liste = []
    for i in serie:
        #i = i.strftime("%Y-%m-%d")
        y = i[:4]
        m = i[5:7]
        d = i[8:]
        i = f"{y}{m}{d}"
        i = int(i)
        Liste.append(i)

    return Liste

def Transform_date_MonthYear(serie):
    Liste = []
    for i in serie:
        i = str(i)
        y = i[:4]
        m = i[4:6]
        i = f'{y}{m}'
        Liste.append(i)

    return Liste

# Différence entre 2 dates en jour:
def difference_dates(date1, date2):
    return abs(date2-date1).days

# Transformation datetime:
def TransformationToDate(serie):
    new_serie = [datetime.strptime(i, "%d-%m-%Y") for i in serie]
    return new_serie

# Affichage Plot avec 3 axes y sur Logs:
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)

# Affichage Plot Aggregate sur Logs:
def plot_aggregate(dataset, churn):
    fig, host = plt.subplots(figsize = (15,6))
    fig.subplots_adjust(right=0.75)

    par1 = host.twinx()
    par2 = host.twinx()

    par2.spines["right"].set_position(("axes", 1.2))

    make_patch_spines_invisible(par2)

    par2.spines["right"].set_visible(True)

    p1, = host.plot(dataset['utilisateur']['2017-03-01':'2017-03-30'].tolist(), label='Nombre Utilisateur', 
             lw = 3, alpha = 0.5, color = 'blue')
    p2, = par1.plot(dataset['sum']['2017-03-01':'2017-03-30'].tolist(), label='Somme d\'écoute par jour', 
             lw = 3, ls = '--', alpha = 0.5, color = 'red')
    p3, = par2.plot(dataset['mean']['2017-03-01':'2017-03-30'].tolist(), label='Moyenne d\'écoute par jour', 
             lw = 2, ls = ':', alpha = 0.5, color = 'purple')

    host.set_xlabel("Date")
    host.set_ylabel('Nombre Utilisateur', color='blue')
    par1.set_ylabel('Somme Temps d\'écoute en seconde', color='red')
    par2.set_ylabel('Moyenne Temps d\'écoute en seconde', color='purple')

    tkw = dict(size=4, width=1.5)
    host.tick_params(axis='y', colors=p1.get_color(), **tkw)
    par1.tick_params(axis='y', colors=p2.get_color(), **tkw)
    par2.tick_params(axis='y', colors=p3.get_color(), **tkw)
    host.tick_params(axis='x', **tkw)

    lines = [p1, p2, p3]

    host.legend(lines, [l.get_label() for l in lines])

    plt.title(f'Analyse temporelle sur les utilisateurs {churn}')
    plt.show()

# Pie Chart Proportion Churn:
def proportion_churn(dataset):
    labels = ['Abonnés', 'Désabonnés']
    colors = [blue, red]
    explode = (0, 0.2)

    fig, ax = plt.subplots()
    ax.pie(dataset['is_churn'].value_counts(), startangle=90, colors=colors, wedgeprops={'edgecolor': 'black'}, autopct='%1.f%%', explode = explode, shadow=True)
    ax.set_title('Proportion d\'attrition', fontweight='bold')

    fig.set_figheight(5)
    fig.set_figwidth(5)
    fig.legend(loc='lower right', labels=labels, fontsize='medium')
    fig.tight_layout()

# Proportion Churn:
def ratio_churn(dataset, name):
    ratio = dataset['is_churn'].value_counts()[1]/len(dataset['is_churn'])
    print(f'Proportion désabonnés sur {name} :\t{round(ratio*100,2)} %')
    
# Proportion Churn Bar:    
def plot_churn(dataset, feature):
    ChurnOnFeature = dataset.groupby([feature, 'is_churn']).size().unstack()
    ChurnOnFeature.rename(columns={0:'Abonnés', 1:'Désabonnés'}, inplace = True)

    colors = ['#8198fd','#fd8181']
    ax = (ChurnOnFeature.T*100.0/ChurnOnFeature.T.sum()).T.plot(kind='bar', width = 0.3, stacked = True, rot = 90, figsize = (15, 4), color = colors, grid=False)

    plt.ylabel('Proportion de Clients\n', horizontalalignment="center", fontstyle = "normal", fontsize="large", fontfamily = "sans-serif")
    plt.xlabel(feature+'\n', horizontalalignment="center", fontstyle = "normal", fontsize="large", fontfamily = "sans-serif")
    plt.title(f'Répartition de l\'Attrition sur {feature}\n', horizontalalignment="center", fontstyle = "normal", fontsize="22", fontfamily = "sans-serif")
    plt.legend(loc='upper right', fontsize = "medium")
    plt.xticks(rotation=0, horizontalalignment="center")
    plt.yticks(rotation=0, horizontalalignment="right")

    ax.yaxis.set_major_formatter(mtick.PercentFormatter())

## Construction des tables Echantillon Base de données principale
### Lecture des CSV

In [4]:
# Logs:
df_logs = pd.read_csv('user_logs_v2.csv')
logs_copy = df_logs.copy()
print(f"Nombre de ligne total :\t\t {len(logs_copy.msno)}")
print(f"Nombre d'utilisateur unique :\t {len(logs_copy.msno.unique())}")
logs_copy.head()

Nombre de ligne total :		 18396362
Nombre d'utilisateur unique :	 1103894


,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,20170331,8,4,0,1,21,18,6309.273
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,20170330,2,2,1,0,9,11,2390.699
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,20170331,52,3,5,3,84,110,23203.337
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,20170331,176,4,2,2,19,191,7100.454
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,20170331,2,1,0,1,112,93,28401.558


In [5]:
# Transactions:
df_transactions = pd.read_csv('transactions_v2.csv')
transactions_copy = df_transactions.copy()
print(f"Nombre de ligne total :\t\t {len(transactions_copy.msno)}")
print(f"Nombre d'utilisateur unique :\t {len(transactions_copy.msno.unique())}")
transactions_copy.head()

Nombre de ligne total :		 1431009
Nombre d'utilisateur unique :	 1197050


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,20170131,20170504,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,20150809,20190412,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,20170329,20170331,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,20170323,20170423,0


In [6]:
# Members:
df_members = pd.read_csv('members_v3.csv')
members_copy = df_members.copy()
print(f"Nombre de ligne total :\t\t {len(members_copy.msno)}")
print(f"Nombre d'utilisateur unique :\t {len(members_copy.msno.unique())}")
members_copy.head()

Nombre de ligne total :		 6769473
Nombre d'utilisateur unique :	 6769473


,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


In [7]:
# Train:
df_train = pd.read_csv('train_v2.csv')
train_copy = df_train.copy()
print(f"Nombre de ligne total :\t\t {len(train_copy.msno)}")
print(f"Nombre d'utilisateur unique :\t {len(train_copy.msno.unique())}")
train_copy.head()

Nombre de ligne total :		 970960
Nombre d'utilisateur unique :	 970960


,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1


In [8]:
# Réduction de l'ID utilisateur msno à 12 caractères:

logs_copy['msno'] = [i[10:22] for i in logs_copy['msno']]
print("Logs :")
print(f"Nombre d'utilisateur unique :\t {len(df_logs.msno.unique())}")
print(f"Nombre d'utilisateur unique :\t {len(logs_copy.msno.unique())}\n")

transactions_copy['msno'] = [i[10:22] for i in transactions_copy['msno']]
print("Transactions :")
print(f"Nombre d'utilisateur unique :\t {len(df_transactions.msno.unique())}")
print(f"Nombre d'utilisateur unique :\t {len(transactions_copy.msno.unique())}\n")

members_copy['msno'] = [i[10:22] for i in members_copy['msno']]
print("Members :")
print(f"Nombre d'utilisateur unique :\t {len(df_members.msno.unique())}")
print(f"Nombre d'utilisateur unique :\t {len(members_copy.msno.unique())}\n")

train_copy['msno'] = [i[10:22] for i in train_copy['msno']]
print("Train :")
print(f"Nombre d'utilisateur unique :\t {len(df_train.msno.unique())}")
print(f"Nombre d'utilisateur unique :\t {len(train_copy.msno.unique())}\n")

Logs :
Nombre d'utilisateur unique :	 1103894
Nombre d'utilisateur unique :	 1103894

Transactions :
Nombre d'utilisateur unique :	 1197050
Nombre d'utilisateur unique :	 1197050

Members :
Nombre d'utilisateur unique :	 6769473
Nombre d'utilisateur unique :	 6769473

Train :
Nombre d'utilisateur unique :	 970960
Nombre d'utilisateur unique :	 970960



## Echantillonnage de démonstration
Cette étape présente la marche à suivre pour l'intégration des CSV dans une base de données relationnelle. Le nombre d'instance étant conséquent, **1%** des données sont introduites dans la BDD pour présenter les requêtes de création et d'introduction de la donnée. L'application souhaitant démontrer l'envoie de requêtes instantanés à travers différentes rubriques, le chargement prendrait du temps. Une seconde base de données a été créée avec des données prétraitées (réduisant le temps d'exécution et d'affichage des résultats dans l'application).

La méthode *Train_Test_Split* est réalisée pour récupérer **1%** des données sur le fichier Train, contenant des ID client unique (*msno*). La séparation est faites de telle sorte que le ratio de la variable cible *is_churn* soit concervé. Les variables temporelles (format *str*) observées en amont sont converties en *datetime* pour leur future insertion en BDD.

![schema_1](Ressources_NB\schema_1.png)

In [9]:
# Split:
r =  40
Train_sample, REM = train_test_split(train_copy, train_size=0.01, stratify = train_copy['is_churn'], random_state = r)
sample_user = Train_sample['msno'].unique()
print(f'Liste d\'utilisateurs uniques : {len(sample_user)}')

# Echantillonnage logs:
logs = logs_copy[logs_copy['msno'].isin(sample_user)]
print(f'Taille pour Logs : {len(logs)}')

# Echantillonnage transactions:
transactions = transactions_copy[transactions_copy['msno'].isin(sample_user)]
print(f'Taille pour Transactions : {len(transactions)}')

# Echantillonnage members:
members = members_copy[members_copy['msno'].isin(sample_user)]
print(f'Taille pour Members : {len(members)}')

# Echantillonnage train:
train = train_copy[train_copy['msno'].isin(sample_user)]
print(f'Taille pour Train : {len(train)}')

# Conversion des variables dates en vu de leur intégration en base SQL:
intermate_date1 = []
for i in logs['date']:
    s = str(i)
    date = datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8])).strftime('%Y:%m:%d')
    intermate_date1.append(date)

logs['date'] = intermate_date1

intermate_date2 = []
for i in transactions['transaction_date']:
    s = str(i)
    date = datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8])).strftime('%Y:%m:%d')
    intermate_date2.append(date)

transactions['transaction_date'] = intermate_date2

intermate_date3 = []
for i in transactions['membership_expire_date']:
    s = str(i)
    date = datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8])).strftime('%Y:%m:%d')
    intermate_date3.append(date)

transactions['membership_expire_date'] = intermate_date3

intermate_date4 = []
for i in members['registration_init_time']:
    s = str(i)
    date = datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8])).strftime('%Y:%m:%d')
    intermate_date4.append(date)

members['registration_init_time'] = intermate_date4

Liste d'utilisateurs uniques : 9709
Taille pour Logs : 137644
Taille pour Transactions : 11388
Taille pour Members : 8629
Taille pour Train : 9709


## Labelling ID User
Pour faire la connexion entre les différentes tables, un ID de type *integer* est attribué pour chaque identifiant *msno* unique. La nouvelle variable est donc mergée avec les DataFrames *logs*, *transaction* et *members*. 

![schema_2](Ressources_NB\schema_2.png)

In [10]:
le = LabelEncoder()
le.fit(train['msno'].tolist())
train['ID_user'] = le.transform(train['msno'].tolist())
train['ID_user'] = [i+1 for i in train['ID_user']]

print(f'Utilisateurs uniques sur df_user : {len(train.ID_user.unique())}')

df_user = train[['ID_user', 'msno', 'is_churn']]
df_user = df_user.sort_values(by=['ID_user'], ascending = True)
df_user = df_user.reset_index(drop=True)

# Merging Transactions ID_USER:
transactions = pd.merge(df_user, transactions, on='msno', how='inner')
transactions = transactions.drop(['is_churn'], 1)

# Merging Logs ID_USER:
logs = pd.merge(df_user, logs, on='msno', how='inner')
logs = logs.drop(['is_churn'], 1)

# Merging Members ID_USER, members contiendra aussi la variable cible is_churn:
members = pd.merge(df_user, members, on='msno', how='inner')

Utilisateurs uniques sur df_user : 9709


## Exports

### Création BDD C2.
Cette BDD relationnelle représente les 4 fichiers CSV d'origine. L'ensemble des données peuvent être inséré directement par les requêtes suivantes (ici comme mentionné, 0.5% des utilisateurs sont introduits, mais le schéma reste le même).

![schema_3](Ressources_NB\schema_3.png)

### Création des Tables C10.
Ces tables reprennent la structure des 4 fichiers CSV. Une table User comprend les Identifiants *msno* et chiffré nommé *User_ID* qui fait le lien avec les autres tables. Les 3 autres tables comprennent respectivement les données de *logs*, *transactions* et *members*. Cette dernière table contient aussi la *target is_churn*.

### Importation C3.
L'ensemble des requêtes pour inserer les données dans la BDD relationnelle type MySQL.

### Export des tables C4.
L'ensemble des requêtes qui font appellent aux données présentes dans la BDD relationnelle afin de les retransformer en DataFrame de même configuration que les fichiers CSV de départ. Une jointure est effectuée avec la table User pour afficher l'identifiant *msno* plutôt que l'identifiant chiffré *User_ID*. Ces requêtes permettent l'analyse théorique des données (si toutefois, l'ensemble des données étaient dans la BDD).

![schema_4](Ressources_NB\schema_4.png)

**L'export des tables en Dataframes permet l'analyse de ces dernières**

In [17]:
# Connexion à la BDD:
config = {
          'user': 'root',
          'password': 'root',
          'host': 'localhost',
          'port': '3307',
          'database': 'DBPCO', 
        }

# Création du Cursor
link = mysql.connector.connect(**config)
cursor = link.cursor(buffered=True)

In [18]:
# Requêtes formation dataframe Transactions:
query = """ SELECT 
                    Transactions.payment_method_id, 
                    Transactions.payment_plan_days, 
                    Transactions.plan_list_price,
                    Transactions.actual_amount_paid, 
                    Transactions.is_auto_renew, 
                    Transactions.transaction_date, 
                    Transactions.membership_expire_date,
                    Transactions.is_cancel, 
                    User.msno
            
            FROM Transactions JOIN User on Transactions.User_ID = User.User_ID
        """
cursor.execute(query)
rows_transactions = cursor.fetchall()

Transactions_sql = []
for values in rows_transactions :
    Transactions_sql.append(list(values))

transaction_columns = ['payment_method_id', 'payment_plan_days', 'plan_list_price','actual_amount_paid', 
                       'is_auto_renew', 'transaction_date', 'membership_expire_date','is_cancel', 'msno']
transaction_export = pd.DataFrame(Transactions_sql, columns = transaction_columns)

In [19]:
# Requêtes formation dataframe Logs:
query = """ SELECT 
                    Logs.date, 
                    Logs.num_25, 
                    Logs.num_50, 
                    Logs.num_75, 
                    Logs.num_985, 
                    Logs.num_100, 
                    Logs.num_unq, 
                    Logs.total_secs, 
                    User.msno
            
            FROM Logs JOIN User on Logs.User_ID = User.User_ID
        """
cursor.execute(query)
rows_logs = cursor.fetchall()

Logs_sql = []
for values in rows_logs :
    Logs_sql.append(list(values))

log_columns = ['date', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq', 'total_secs', 'msno']
log_export = pd.DataFrame(Logs_sql, columns = log_columns)

In [20]:
# Requêtes formation dataframe members:
query = """ SELECT 
                    Members.city, 
                    Members.bd, 
                    Members.gender, 
                    Members.registered_via, 
                    Members.registration_init_time,
                    Members.is_churn,
                    User.msno
            
            FROM Members JOIN User on Members.User_ID = User.User_ID
        """
cursor.execute(query)
rows_members = cursor.fetchall()
link.close()

Members_sql = []
for values in rows_members :
    Members_sql.append(list(values))

member_columns = ['city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'is_churn', 'msno']
member_export = pd.DataFrame(Members_sql, columns = member_columns)

## Preprocessing
Le modèle créé en amont fonctionnait sur des données ayant subit certains traitements (réduction des valeurs aberrantes, suppression de données manquantes, regroupement etc...). Les scripts présentent un condensé du travail de ***preprocessing*** effectué durant l'analyse générale.

### Logs
Représentant le nombre de musique et le temps d'écoute journalier pour chaque utilisateur, les informations ont été reportées sur le mois. Les nouvelles variables correspondent à la somme et moyenne d'écoute/musiques écoutées. Une variable *count* repertorie le nombre de jour de connexion de chaque utilisateur.

Les valeurs aberrantes ont été détectées en appliquant un seuil (Z-score). Ces données ont été changées en la médiane de chaque variable.

In [21]:
# Processing Logs:

Features = [log_export.num_25, log_export.num_50, log_export.num_75, log_export.num_985, log_export.num_100, 
            log_export.num_unq, log_export.total_secs]
name_features = ['num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq', 'total_secs']

Outliers_del = []

for i, name in zip(Features, name_features) :
    Identify_outliers = []
    
    # Calcul du Z Score
    log_export['Z_score']=(i-i.mean())/i.std()
    z = len(log_export[(log_export['Z_score']>3)])
    
    # Identification des valeurs aberrantes
    Outliers_del.append(log_export[(log_export['Z_score']>3) | (log_export['Z_score']<-3)].shape[0])
    Identify_outliers = log_export[(log_export['Z_score']>3) | (log_export['Z_score']<-3)].index
    R_with_Median = log_export[(log_export['Z_score']<3) & (log_export['Z_score']>-3)][name].median()
    
    # Remplacement par la médiane
    log_export.loc[Identify_outliers, name] = R_with_Median
    
log_export = log_export.drop(['Z_score'], 1)

# Regroupement des informations sur le mois, une instance correspond à un utilisateur unique:
LogsSum = log_export.groupby('msno').sum().reset_index()
LogsSum.rename(columns={'num_25':'num_25Sum','num_50':'num_50Sum','num_75':'num_75Sum','num_985':'num_985Sum',
                        'num_100':'num_100Sum','num_unq':'num_unqSum','total_secs':'total_secsSum'}, inplace = True)

LogsMean = log_export.groupby('msno').mean().reset_index()
LogsMean.rename(columns={'num_25':'num_25Mean','num_50':'num_50Mean','num_75':'num_75Mean','num_985':'num_985Mean',
                         'num_100':'num_100Mean','num_unq':'num_unqMean','total_secs':'total_secsMean'}, inplace = True)

days_count = log_export.groupby('msno').size().to_frame('count')
days_count = pd.merge(log_export, days_count, on='msno', how='inner')
days_count = days_count[['msno', 'count']]
days_count = days_count.groupby('msno').mean()

Preprocess_logs = pd.merge(LogsSum, LogsMean, on='msno', how='inner')
Preprocess_logs = pd.merge(Preprocess_logs, days_count, on='msno', how='inner')

### Transactions
La variable représentant le nombre de transaction sur le mois a été créée et ajouté au DataFrame, au vu d'un futur report sur le mois.

In [22]:
# Nombre de transactions par utilisateur:
transaction_count = transaction_export.groupby('msno').size().to_frame('transaction_count')
transaction_count = pd.merge(transaction_export, transaction_count, on='msno', how='inner')
transaction_count = transaction_count[['msno', 'transaction_count']]
transaction_count = transaction_count.groupby('msno').mean()

Preprocess_transaction = pd.merge(transaction_export, transaction_count, on='msno', how='inner')

### Members
Les valeurs manquantes de départ sur la variable *gender*, concervé dans la base de données, ont été remplacées par le terme "inconnu".

Pour l'âge, les valeurs aberrantes situées en dehors d'un intervalle raisonnable de 10 à 70 ans ont été fixé à -1. Le choix d'un remplacement plutôt qu'une suppression s'explique par une variabilité certaine entre les utilisateurs restants abonnés et les utilisateurs désabonnés.

In [23]:
# Remplacement des valeurs manquantes de la variable gender par 'inconnu'
member_export['gender'] = member_export['gender'].replace(to_replace ="nan", value ="inconnu")

# Nettoyage des valeurs abérrantes:
member_export['bd'] = [i if 10 < i < 70 else -1 for i in member_export['bd']]

### Merge
Les tables "converties" en Dataframes sont fusionnées. De nouvelles variables sont créées à l'occasion en effectuant des rapports entre plusieurs variables etc... Les données de variables temporelles sont converties en *integer*. Finalement, chaque instance correspond à un utilisateur unique, après l'application d'un filtre retenant, la dernière date de transaction effectuée.

Dans le cadre de ce projet, c'est ce genre de Dataframe qui a été introduit dans une autre BDD relationnelle connectée à l'application de démonstration.

![schema_5](Ressources_NB\schema_5.png)

### Base de données relationnelle avec preprocessing sur 43544 Utilisateurs uniques (Dataset Test)
**BDD utilisée pour la démonstration de l'application**
![schema_6](Ressources_NB\schema_6.png)

In [24]:
# Merging et Groupby msno sur membership_expire_date:
Dataset = pd.merge(Preprocess_transaction, member_export, on='msno', how='inner')
Dataset = pd.merge(Dataset, Preprocess_logs, on='msno', how='inner')


# Feature Engineering___ Prix abonnement / Temps abonnement:
Price_per_day = []
for i, j in zip(Dataset.plan_list_price, Dataset.payment_plan_days):
    try:
        k = i/j
        Price_per_day.append(k)
    except:
        Price_per_day.append(0)

Dataset['price_per_day'] = Price_per_day

# Feature Engineering___ Différences entre membership_expire_date et registration_init_time en jour:
Dataset['days_fidelity'] = [difference_dates(i, j) for i, j in zip(Dataset.membership_expire_date, Dataset.registration_init_time)]

# Reconversion des Dates en INT:
Dataset['membership_expire_date'] = [int(i.strftime("%Y%m%d")) for i in Dataset['membership_expire_date']]
Dataset['registration_init_time'] = [int(i.strftime("%Y%m%d")) for i in Dataset['registration_init_time']]
Dataset['transaction_date'] = [int(i.strftime("%Y%m%d")) for i in Dataset['transaction_date']]

# Réduction sur la dernière date de transaction:
Dataset = Dataset.loc[Dataset.groupby('msno')['membership_expire_date'].idxmax()]

print(f'Nombre d\'utilisateurs unique : {len(Dataset.msno.unique())} pour {len(Dataset)} lignes')

Nombre d'utilisateurs unique : 7323 pour 7323 lignes


### Preprocessing for Model
La démarche est similaire que lors de la création des modèles IA. Le Dataframe final est traité une deuxième fois pour encoder les variables catégorielles telles que *gender*, *payment_method_id*, *registered_via*, *city*. Par la suite, l'entrainement du modèle peut commencé si, bien entendu, la BDD créé ici contenait l'ensemble des données.

L'analyse et la modélisation ayant été effectuées en amont directement à partir des fichiers CSV, plusieurs fichiers Joblib ont été enregistré. Ils comprennent :
* Le modèle XGBoost Classifier Optimisé après Feature Sélection.
* L'échelle de Standardisation *fit* sur les données Train et permettant de transformer de nouvelles données.
* ColonneScaler, une liste des variables présentent lors de la Standardisation des données d'entraînement.
* ColonnesModel, une liste des variables sélectionnés (*feature_selection*) pour les prédictions.

In [26]:
# Importation des Fichiers de Modèles et Scaler:
scaler = joblib.load('ProcessModel/ScalerXGBC_BF.joblib')
colonnesmodel = joblib.load('ProcessModel/ColonnesXGBC_BF.joblib')
classifier = joblib.load('ProcessModel/XGBC_BF.joblib')
colonnescaler = joblib.load('ProcessModel/ColonnesForScale.joblib')

In [27]:
# Modification de series catégorielles:
df = Dataset.copy()

df['city'] = [str(i)+'C' for i in df.city]
df['payment_method_id'] = [str(i)+'P' for i in df.payment_method_id]
df['registered_via'] = [str(i)+'R' for i in df.registered_via]

# Encodage:
df = pd.concat([df, pd.get_dummies(df.gender)],1)
df = pd.concat([df, pd.get_dummies(df.payment_method_id)],1)
df = pd.concat([df, pd.get_dummies(df.registered_via)],1)
df = pd.concat([df, pd.get_dummies(df.city)],1)
df = df.drop(['gender', 'payment_method_id','registered_via', 'city'], 1)


# Calibration avec le dataset de départ ayant servi à la modélisation: 
for feature in colonnescaler:
        if feature not in df.columns:
            df[feature]=0
            
df = df[colonnescaler]

# Vérification:
find_NaN(df)

Aucune valeur manquante n'est détectée sur ce dataset


## Traitement du modèle (Démonstration NB)
Démontre que la suite des processus, débutant sur l'ensemble des requêtes pour la création des nouveaux Dataframes depuis la BDD jusqu'aux travaux de prétraitements forme un jeu de donnée cohérent et pouvant être passé dans le modèle IA créé en amont sans erreur.

In [28]:
X_test = []
X_test = df.drop(['msno','is_churn'], axis = 1)
y_test = df['is_churn']

# Standardisation des données Test avec le scaler fit sur données d'entrainement pour l'intégration Web
X_test2 = pd.DataFrame(scaler.transform(X_test.values))
X_test2.columns = X_test.columns.values
X_test2.index = X_test.index.values
X_test = X_test2

X_test = X_test[colonnesmodel]

y_pred = classifier.predict(X_test)
probability = classifier.predict_proba(X_test)
probability = probability[:,1]

final_results = df
final_results['predictions'] = y_pred
final_results['membership_expire_date'] = df['membership_expire_date']
final_results["propensity_to_churn(%)"] = probability
final_results["propensity_to_churn(%)"] = final_results["propensity_to_churn(%)"]*100
final_results["propensity_to_churn(%)"] = final_results["propensity_to_churn(%)"].astype(int)
final_results = final_results.sort_values(by=['propensity_to_churn(%)'], ascending = False)
Results = final_results[['msno', 'is_churn', 'propensity_to_churn(%)']]
Results.index.msno=None

### Résultats
Application sommaire du modèle IA sur le Dataset construit à partir de cette BDD relationnelle.

In [29]:
auc = roc_auc_score(y_test, probability)
logloss = log_loss(y_test, probability)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

print(f"""AUC :\t\t{round(auc,2)}\nLogloss :\t{round(logloss,2)}\nF1 :\t\t{round(f1,2)}\nPrecision :\t{round(precision,2)}\nRecall :\t{round(recall,2)}\nMCC :\t\t{round(mcc,2)}""")

AUC :		0.99
Logloss :	0.06
F1 :		0.83
Precision :	0.95
Recall :	0.74
MCC :		0.83


In [30]:
# Echantillon des prédictions obtenues:
Table = pd.DataFrame(columns = Results.columns)
for i in range (10, 110, 10):
    j = i - 10
    Filter = Results.loc[Results['propensity_to_churn(%)'] < i]
    Sample = Filter.loc[Filter['propensity_to_churn(%)'] > j].sample(n = 3)
    Table = Table.append(Sample)
    
Table = Table.sort_values(by = ['propensity_to_churn(%)'], ascending = [False])
Table.style.background_gradient(cmap="Reds", subset=['propensity_to_churn(%)'])

,msno,is_churn,propensity_to_churn(%)
5094,Xdk6lr1w+CTJ,1,99
1979,BCm4b9AW1ZP3,1,99
5005,X9gUkD598ov0,1,98
8322,uQ5rnorFRfTY,1,89
7260,mZFZzOn+yH4F,1,81
2589,FeuDXL3Kcb50,1,81
6336,gJhiiGWz7GQU,1,77
382,0s4t24vcHNYC,1,75
5803,crz21yx2XVnS,1,74
72,+YfwO9pqidtb,1,63
